In [1]:
import numpy as np
import math
import random

In [6]:
G=1000
nstrat=12             #types d'agents
I=100                 #nbre d'agent par strat
K= nstrat*I           #taille du marché
Lr=10                 #taille marché réduit
N=100                 #nbre de périodes d'intéraction sur le marché réduit

alpha=0.1             #seuil de déclenchement
vp=0.05               #variation prix


A=336.78              #paramètres (macro) du modèle
a=3.18*0.00001
b=1
qo=100
CF=30

Pp=1.313              #prudent
Qp=188  
Pcart=2.037           #cartel
Qcart=82  
Pcon=1.307            #équilibre marché
Qcon=198
sigma=0.02

Pnew=1.087             #bidouillage pour que les coopératifs s'imposent (tout en restant coopératifs)
Qnew=284.9             #(prix,qté) qui permet d'égaliser la demande indiv au cout marginal



def demande(p):
    return Lr*A/(p*p)

def demande_indiv(p):
    return A/(p*p)

def CT(Q):
    return (a*Q*Q*Q*(1/3)-a*qo*Q*Q+a*(qo^2+b)*Q+CF)

def profits(M):
    demande_iter=0
    prof=np.zeros(Lr)
    for i in range(Lr):
        prix=M[i,1]
        Q_prod=M[i,2]
        Q_ecoul=min((demande(prix)-demande_iter),Q_prod)
        prof[i]=prix*Q_ecoul-CT(Q_prod)
        demande_iter+=Q_ecoul
    return prof

def profit_indiv(M,j):
    demande_iter=0
    for i in range(j+1):
        prix=M[i,1]
        Q_prod=M[i,2]
        Q_ecoul=min((demande(prix)-demande_iter),Q_prod)
        demande_iter+=Q_ecoul
    prof_j=prix*Q_ecoul-CT(Q_prod)
    return prof_j

def ratio(M):                         #retourne le rapport invendus sur qté produite
    demande_iter=0
    rat=np.zeros(Lr)
    for i in range(Lr):
        Q_ecoul=min((demande(M[i,1])-demande_iter),M[i,2])
        rat[i]=max(M[i,2]-max(0,(demande(M[i,1])-demande_iter)),0)/M[i,2]
        demande_iter+=Q_ecoul
    return rat


#création du marché initial/global:

def initialisation_marche(K):
    h=np.ones(K)
    for i in range(1,nstrat+1):
        h[I*i:K]+=np.ones(K-I*i)
    return h




#marché réduit issu d'un tirage aléatoire sur le marché global
#complété par les prix,qtés (avec aléa),profits,profits cumulés.

def marche_reduit(M,Lr):
    hr=random.sample(list(M),Lr)     #marché réduit issu d'un tirage aléatoire sur le marché global (liste de types)
    
    Mr= np.transpose(np.array([hr,np.ones(Lr)*Pcon,np.ones(Lr)*Qcon,np.zeros(Lr),np.zeros(Lr)]))

    s1 = np.random.normal(0, sigma,Lr)
    for i in range(Lr):
        s1[i]=max(s1[i],-2*sigma)
        s1[i]=min(s1[i],2*sigma)
    s2 = np.random.normal(0, sigma,Lr)
    for i in range(Lr):
        s2[i]=max(s2[i],-2*sigma)
        s2[i]=min(s2[i],2*sigma)
        Mr[:,1:3]=Mr[:,1:3]*(1+np.transpose(np.array([s1,s2])))     # aléa sur les prix et prod:

    return Mr




#définition des différentes stratégies qui sont associées à un type

def coop_1():
    return np.array([Pcart,Qcart])

def coop_2(ration_indiv,p,q):
    if (ration_indiv<=alpha):
        p=min(Pcart,p+vp*(Pcart-Pcon))
    else:
        p=max(Pcon,p+vp*(Pcon-Pcart))
    q=demande_indiv(p)
    return np.array([p,q])

def coop_3(marche,j,ration,p,q):
    if ration[j] <= 0 :
        p = p*(1+vp)
    else:
        i=0
        while ration[i]<=0:
            i=i+1
        p = marche[i,1]*(1-vp)
        q = demande_indiv(p)
    return np.array([p,q])
        

def coop_4(ration):
    if (np.mean(ration)<=alpha):
        return np.array([Pcart,Qcart])
    else:
        return np.array([Pp,Qp])
    
def strat_1(marche,j,ration):
    Mtest1 = marche
    Mtest2 = marche
    Mtest1[j,1:3] = np.array([Pcart,Qcart])
    order = Mtest1[:,1].argsort()
    Mtest1 = np.take(Mtest1, order, 0)
    i=0
    while (ration[i]<=0 and i<Lr-1):
        i=i+1
    p = marche[i,1]*(1-vp)
    q = qo + math.sqrt((p-a)/a)
    Mtest2[j,1:3] = np.array([p,q])
    order = Mtest2[:,1].argsort()
    Mtest2 = np.take(Mtest2, order, 0)
    profit1 = profit_indiv(Mtest1,j)
    profit2 = profit_indiv(Mtest2,j)
    if profit1 >= profit2:
        return np.array([Pcart,Qcart])
    else:
        return np.array([p,q])
    

def strat_2(marche,j,ration):
    if ration[j] <= alpha:
        return(strat_1(marche,j,ration))
    else:
        p = marche[j,1]*(1-vp)
        q = marche[j,2]
        return np.array([p,q])

def strat_3(marche,j,ration):
    if ration[j] <= alpha:
        p = marche[j,1]*(1+vp)
        q = marche[j,2]        
    else:
        i=0
        while (ration[i]<=0):
            i=i+1
        p = marche[max(0,i-1),1]
        q = qo+math.sqrt((p-a)/a)
    return np.array([p,q])
        
def strat_4(marcheprec,j,ration):
    if marcheprec[j,4]==0:
        return(np.array([Pcon,Qcon]))
    PQ = strat_1(marcheprec,j,ration)
    PQ[0] = PQ[0]*(1-vp)
    PQ[1] = qo + math.sqrt((PQ[0]-a)/a)
    return PQ

def pdp_1():
    return(Pcon,Qcon)

def pdp_2(marche):
    prix=marche[:,1]
    p=np.mean(prix)
    q = qo+math.sqrt((p-a)/a)
    return np.array([p,q])

def pdp_3(marche):
    order = marche[:,3].argsort()
    marche_ordo = np.take(marche, order, 0)
    p = marche_ordo[0,1]
    q = qo+math.sqrt((p-a)/a)
    return np.array([p,q])

def pdp_4(marche,j,ration_indiv):
    p=marche[j,1]
    q=marche[j,2]
    if ration_indiv<=alpha:
        p=(1+vp)*p
    else:
        p=(1-vp)*p
    return np.array([p,q])

    
    



#intéractions sur le marché réduit

def interactions_marche(Mr,Lr,N):
    
    
    for i in range(N):
        order = Mr[:,1].argsort()         #marché trié par prix
        Mr=np.take(Mr, order, 0)
        
        Mrprec=Mr
        
        Mr[:,3]=profits(Mr)
        Mr[:,4]+=Mr[:,3]
        
        vect_rationnement=ratio(Mr)
        copyMr=Mr
        for j in range(Lr):
            if Mr[j,0]==1:
                Mr[j,1:3]=coop_1()
            if Mr[j,0]==2:
                Mr[j,1:3]=coop_2(vect_rationnement[j],Mr[j,1],Mr[j,2])
            if Mr[j,0]==3:
                Mr[j,1:3]=coop_3(copyMr,j,vect_rationnement,Mr[j,1],Mr[j,2])        
            if Mr[j,0]==4:
                Mr[j,1:3]=coop_4(vect_rationnement)
            if Mr[j,0]==5:
                Mr[j,1:3]=strat_1(copyMr,j,vect_rationnement)
            if Mr[j,0]==6:
                Mr[j,1:3]=strat_2(copyMr,j,vect_rationnement)
            if Mr[j,0]==7:
                Mr[j,1:3]=strat_3(copyMr,j,vect_rationnement)
            if Mr[j,0]==8:
                Mr[j,1:3]=strat_4(Mrprec,j,vect_rationnement)
            if Mr[j,0]==9:
                Mr[j,1:3]=pdp_1()
            if Mr[j,0]==10:
                Mr[j,1:3]=pdp_2(copyMr)
            if Mr[j,0]==11:
                Mr[j,1:3]=pdp_3(copyMr)
            if Mr[j,0]==12:
                Mr[j,1:3]=pdp_4(copyMr,j,vect_rationnement[j])
                
        s1 = np.random.normal(0, sigma,Lr)
        for i in range(Lr):
            s1[i]=max(s1[i],-2*sigma)
            s1[i]=min(s1[i],2*sigma)
        s2 = np.random.normal(0, sigma,Lr)
        for i in range(Lr):
            s2[i]=max(s2[i],-2*sigma)
            s2[i]=min(s2[i],2*sigma)
        Mr[:,1:3]=Mr[:,1:3]*(1+np.transpose(np.array([s1,s2])))
        order = Mr[:,1].argsort()
        Mr=np.take(Mr, order, 0)
                
    Mr[:,3]=profits(Mr)
    Mr[:,4]+=Mr[:,3]
    
    

    return(Mr)

#détermination de l'efficacité de chaque agent (fonction fitness) à partir des resultats du marché

def efficacite(Mr):
        order = Mr[:,4].argsort()
        Mr=np.take(Mr, order, 0)                    #marché trié par profits cumulés
        profits_cumul=Mr[:,4]
        profits_cumul=profits_cumul-np.min(profits_cumul)
        eff=np.zeros(Lr)
        types=Mr[:,0]
        eff = np.round(Lr*profits_cumul/np.sum(profits_cumul))
        reprod=np.where(eff>=1)[0]
        ind=0
        while sum(eff)>Lr:       #si l'arrondi est trop grand, on enlève aux effectifs qui ont le moins bien réussi.
            while eff[ind]>0:
                eff[ind]=eff[ind]-1
            ind+=1
        while sum(eff)<Lr:        #si l'arrondi est trop bas, on rajoute à l'effectif de l'agent qui a eu les meilleurs profits
            k=random.randint(reprod[0],reprod[len(reprod)-1])
            eff[k]+=1
        
        return np.array([types,eff])


#processus d'évolution ie modification du marché initial en fonction des efficacités de chaque type dans le marché réduit

def evol_simple(M,efficacite):
        eff=efficacite[1,:]
        elimines=np.where(eff==0)[0]       #liste d'indices agents qui ne survivent pas
        reprod=np.where(eff>1)[0]          #liste d'indices qui corresp aux agents qui vont se "reproduire"
        ind=0
        for i in elimines:
            t=efficacite[0,i]
            if eff[reprod[ind]]==1:
                ind+=1
            M[np.where(M==t)[0][0]]=efficacite[0,reprod[ind]]
            eff[reprod[ind]]=eff[reprod[ind]]-1
        return(M)

#processus répété

def evolution(M,Lr,G):
    for i in range(G):
        Mtest = marche_reduit(M,Lr)
        Mtest = interactions_marche(Mtest,Lr,N)
        eff = efficacite(Mtest)
        M=evol_simple(M,eff)
        if i%200==0:
            X=[]
            for j in range(1,13):
                X.append(np.count_nonzero(M == j))
                
            print(i)
            print(X)
            print(Mtest)

    return M



MARCHE=initialisation_marche(K)
mar=marche_reduit(MARCHE,Lr)
mar=interactions_marche(mar,Lr,N)
ef=efficacite(mar)
print(mar)
print(ef)


Z=evolution(MARCHE,Lr,G)
X=[]
for i in range(1,13):
    X.append(np.count_nonzero(Z == i))
X


[[ 4.00000000e+00  1.28429476e+00  2.24923086e+02  2.98391438e+02
   2.99879248e+04]
 [ 9.00000000e+00  1.30614895e+00  2.04366728e+02  2.78602485e+02
   2.73593602e+04]
 [ 1.00000000e+01  1.84140391e+00  3.35045853e+02  5.44155899e+02
   2.64472166e+04]
 [ 2.00000000e+00  1.92710655e+00  9.66563881e+01  1.76087698e+02
   9.89853585e+03]
 [ 6.00000000e+00  1.93211129e+00  8.42016800e+01  6.54769568e+01
   4.14081505e+03]
 [ 8.00000000e+00  1.93714229e+00  3.42567594e+02 -9.31391739e+01
  -3.55434611e+03]
 [ 6.00000000e+00  2.00949265e+00  7.90979435e+01 -1.43136812e+02
   5.70490112e+03]
 [ 1.00000000e+00  2.02415227e+00  8.30892524e+01 -3.87626245e+01
  -1.41969066e+04]
 [ 6.00000000e+00  2.02886862e+00  8.41137831e+01 -2.18292802e+01
   2.93122707e+03]
 [ 5.00000000e+00  2.11848000e+00  8.06083226e+01 -1.58684963e+02
  -1.95887371e+03]]
[[ 1.  8.  5.  6.  6.  6.  2. 10.  9.  4.]
 [ 0.  0.  0.  1.  1.  1.  1.  2.  2.  2.]]
0
[99, 100, 100, 100, 100, 100, 100, 100, 101, 100, 100, 100]


[0, 1, 3, 506, 1, 0, 0, 0, 323, 364, 1, 1]